<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Amun Inspection Workflow

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys

In [3]:
USER, = !echo "$$USER"

sys.path.insert(0, f"/home/{USER}/code/amun-api/")
sys.path.insert(0, f"/home/{USER}/code/argo-python-client/")
sys.path.insert(0, f"/home/{USER}/code/common/")

---

In [4]:
# Provided via https://github.com/thoth-station/amun-api

from amun.dockerfile import create_dockerfile

In [5]:
from thoth.common.workflows import Workflow
from thoth.common.workflows import WorkflowManager

In [6]:
from thoth.common.logging import init_logging

In [7]:
init_logging({'thoth.common.workflows': 'INFO'})

2019-11-28 10:23:03,487 [11479] WARNING  root:188: Logging to a Sentry instance is turned off
2019-11-28 10:23:03,491 [11479] INFO     root:210: Logging to rsyslog endpoint is turned off


---

In [8]:
namespace = "macermak-thoth-dev"

In [9]:
%env THOTH_AMUN_INFRA_NAMESPACE      $namespace 
%env THOTH_AMUN_INSPECTION_NAMESPACE $namespace
%env THOTH_WORKFLOW_CONTEXT          true

env: THOTH_AMUN_INFRA_NAMESPACE=macermak-thoth-dev
env: THOTH_AMUN_INSPECTION_NAMESPACE=macermak-thoth-dev
env: THOTH_WORKFLOW_CONTEXT=true


In [10]:
workflow_manager = WorkflowManager(ocp_config={
    "kubernetes_verify_tls": False
})

2019-11-28 10:23:06,588 [11479] WARNING  thoth.common.openshift:103: Failed to load in cluster configuration, fallback to a local development setup: Service host/port is not set.
2019-11-28 10:23:07,621 [11479] WARNING  thoth.common.openshift:140: TLS verification when communicating with k8s/okd master is disabled


---

In [11]:
specification = {
  "base": "registry.access.redhat.com/ubi8/python-36",
  "build": {
    "requests": {
      "cpu": "500m",
      "hardware": {
        "cpu_family": 6,
        "cpu_model": 94,
        "physical_cpus": 32, "processor": "Intel-Xeon-Processor-Skylake-IBRS"
      },
      "memory": "1Gi"
    }
  },
  "identifier": "test-inspection",
  "python_packages": [
    "pipenv"
  ],
  "run": {
    "requests": {
      "cpu": "500m",
      "hardware": {
        "cpu_family": 6,
        "cpu_model": 94,
        "physical_cpus": 32,
        "processor": "Intel-Xeon-Processor-Skylake-IBRS"
      },
      "memory": "256Mi"
    }
  },
  "script": "#!/usr/bin/bash\necho \"Here should be run tests...\"\n",
}

In [12]:
dockerfile, _ = create_dockerfile(specification)

In [13]:
inspection_id = "test-inspection-v2"

template_parameters = {
    "AMUN_INSPECTION_ID": inspection_id,
    "THOTH_INFRA_NAMESPACE": namespace
}

workflow_parameters=dict(
    dockerfile=dockerfile,
    specification=json.dumps(specification)
)

In [14]:
# !argo delete --all

workflow_id = workflow_manager.submit_inspection_workflow(
    inspection_id,
    template_parameters=template_parameters,
    workflow_parameters=workflow_parameters,
)

workflow_id

In [16]:
!argo watch $workflow_id

FATA[0000] workflows.argoproj.io "None" not found       


---